In [1]:
import os
import python_debugger as pyd
import javalang
import xml.etree.ElementTree as ET
import numpy as np

In [2]:
def listdir_fullpath(d):
    return [f for f in os.listdir(d)]

In [3]:
def augmentCode(sourcefileX, changeVariable,permuStatement, task=1 ):
    data=pyd.readJavaFile(sourcefileX)
    tree = javalang.parse.parse(data)
    newCode,_, permutation = pyd.AST(tree).getAugmentedCode(task = task, changeVariable=changeVariable, permuStatement=permuStatement)
    return newCode,permutation

In [4]:
def inputGenerate(list_newCode,copyNum,txtfiledir,case):
    copyfiledir = txtfiledir + "\\" + str(copyNum)  #create txt case/0 dir, e.g. XX/CASE/0
    if not os.path.exists(copyfiledir):
        os.mkdir(copyfiledir)
    f_input=open(copyfiledir+"\\"+case+".txt",'w') #寫檔
    for newCode in list_newCode:
        f_input.write(" <BOC> ")
        NEWLINE = 	"abcdefgijkoooopppp"
        for content in newCode:
            if(content==NEWLINE):
                f_input.write(" <CR> ")
            else:
                f_input.write(content)
                f_input.write(" ")
            #print(content)
        f_input.write(" <EOC> ")
    f_input.close()

In [5]:
def changeLineBlock(begin, end, permutation):
    permu_lineblock = org_lineblock = [value for value in range(begin, end+1)]#make permu_lineblock get permutation value    
    for i in range(len(org_lineblock)):
        permu_lineblock[i] = permutation[permu_lineblock[i]]    
    permu_lineblock.sort()#sort permu
    line = [] 
    lines = []
    last_permu = (len(permu_lineblock)-1)
    for i in range(len(permu_lineblock)):
        if (i == 0):
            line.append(permu_lineblock[i])
            if (permu_lineblock[i] == permu_lineblock[last_permu]):
                lines.append(list(line))
                line.clear()
        elif (permu_lineblock[i] == (permu_lineblock[i-1]+1) ):
            line.append(permu_lineblock[i])
            if (permu_lineblock[i] == permu_lineblock[last_permu]):
                lines.append(list(line))
                line.clear()
        elif (permu_lineblock[i] != (permu_lineblock[i-1]+1) ):
            lines.append(list(line))
            line.clear()
            line.append(permu_lineblock[i])
            if (permu_lineblock[i] == permu_lineblock[last_permu]):
                lines.append(list(line))
                line.clear()
        else:
            line.append(permu_lineblock[i])
            lines.append(line)
            line.clear()
    return lines

In [6]:
def xmlGenerate(src_dict,copyNum,xmlfiledir,case):
    copyfiledir = txtfiledir + "\\" + str(copyNum)  #create txt case/0 dir, e.g. XX/CASE/0
    if not os.path.exists(copyfiledir):
        os.mkdir(copyfiledir)
        tree = ET.parse("test.xml")#抓到xml檔
        root = tree.getroot()#找到根節點
        for Linelist in root.iter("Linelist"):# 尋找 Linelist 節點
            for line in Linelist.iter("Line"):#在Linelist 尋找 Line 節點
                old_begin=line.get("Begin")# 取得節點指定屬性質
                old_end=line.get("End")
                src=line.get("src")
                print("src",src)
                line_blocks=changeLineBlock(old_begin,old_end,src_dict["src"])
                for line_block in line_blocks:
                    element=Element('line',{'src':src},{'End':line_block[1]},{'Begin':line_block[0]})
                    Linelist.append(element)
                root.remove(line)
        tree.write("output.xml")

In [8]:
if __name__ == '__main__' :
    source_dir = "D:/NewVerPythonDebugger/NormalizedCODES" #the source codes
    xml_source_dir ="D:/NewVerPythonDebugger/SourceXml"
    dest_dir = "D:/NewVerPythonDebugger/AugmentedCODES" #store the augmented files
    txt_dir = "D:/NewVerPythonDebugger/InputTxt"
    xml_dir = "D:/NewVerPythonDebugger/OutputXml"
    cases = listdir_fullpath(source_dir)
    print("cases",cases)
    xml_cases = listdir_fullpath(xml_source_dir)
    copyNum = 1
    for case in cases:
        list_newCode=[]
        src_dict={}
        sourcefiledir = source_dir + "/" + case
        print("sourcefiledir",sourcefiledir)
        destfiledir = dest_dir + "/" + case
        if not os.path.exists(destfiledir):
            os.mkdir(destfiledir)  #create dest case dir, e.g. XX/CASE
        txtfiledir = txt_dir + "/" + case
        if not os.path.exists(txtfiledir):
            os.mkdir(txtfiledir)  #create txt case dir, e.g. XX/CASE
        xmlfiledir = xml_dir + "/" + case
        if not os.path.exists(xmlfiledir):
            os.mkdir(xmlfiledir)  #create xml case dir, e.g. XX/CASE
        sourcefiles = listdir_fullpath(sourcefiledir)
        for sourcefile in sourcefiles: # .java
            for i in range(copyNum):
                print("Augmenting case : ", case, "sourcefile: ",sourcefile, " copy num: ", i, " ......") 
                copyfiledir = destfiledir + "/" + str(i)  #create dest case/0 dir, e.g. XX/CASE/0
                if not os.path.exists(copyfiledir):
                    os.mkdir(copyfiledir)  #create dest case/0 dir
                destfile = copyfiledir + "/" + sourcefile
                print("destfile:",destfile)
                newCode, permutation = augmentCode(sourcefiledir+"/"+sourcefile, True, True, 1)
                pyd.write_to_file(newCode, destfile)
                
                for sourcefile in sourcefiles:
                newCode, permutation = augmentCode(sourcefiledir+"/"+sourcefile, True, True, 1)
                list_newCode.append(newCode)
                #print("list_newCode",list_newCode)
                src_dict[sourcefile]=permutation
                #print(src_dict[sourcefile])
            inputGenerate(list_newCode,copyNum,txtfiledir,case)
            xmlGenerate(src_dict,copyNum,xmlfiledir,case)
'''
    for case in cases:            
        for i in range(copyNum):
            for sourcefile in sourcefiles:
                newCode, permutation = augmentCode(sourcefiledir+"/"+sourcefile, True, True, 1)
                list_newCode.append(newCode)
                #print("list_newCode",list_newCode)
                src_dict[sourcefile]=permutation
                #print(src_dict[sourcefile])
            inputGenerate(list_newCode,copyNum,txtfiledir,case)
            xmlGenerate(src_dict,copyNum,xmlfiledir,case)
'''        

cases ['2', '792']
sourcefiledir D:/NewVerPythonDebugger/NormalizedCODES/2
Augmenting case :  2 sourcefile:  ChineseLetter.java  copy num:  0  ......
destfile: D:/NewVerPythonDebugger/AugmentedCODES/2/0/ChineseLetter.java
Augmenting case :  2 sourcefile:  EnglishLetter.java  copy num:  0  ......
destfile: D:/NewVerPythonDebugger/AugmentedCODES/2/0/EnglishLetter.java
Augmenting case :  2 sourcefile:  Letter.java  copy num:  0  ......
destfile: D:/NewVerPythonDebugger/AugmentedCODES/2/0/Letter.java
sourcefiledir D:/NewVerPythonDebugger/NormalizedCODES/792
Augmenting case :  792 sourcefile:  Evaluation.java  copy num:  0  ......
destfile: D:/NewVerPythonDebugger/AugmentedCODES/792/0/Evaluation.java
Augmenting case :  792 sourcefile:  Exam.java  copy num:  0  ......
destfile: D:/NewVerPythonDebugger/AugmentedCODES/792/0/Exam.java
Augmenting case :  792 sourcefile:  HomeWork.java  copy num:  0  ......
destfile: D:/NewVerPythonDebugger/AugmentedCODES/792/0/HomeWork.java
Augmenting case :  79